In [1]:
import torch
from t3_karpathy.compressing_transformer import CompressingAccumulativeTrainer
from t3_karpathy.transformer_config import TransformerConfig
from t3_karpathy.gpt_nano_dataloader import GptNanoDataloader
from t3_karpathy.karpathy_transformer import KarpathyRunner



In [2]:

config = TransformerConfig()
dataloader = GptNanoDataloader(config)
device = 'cuda'

trainer1=CompressingAccumulativeTrainer(config)
#rainer1=KarpathyRunner(config)
trainer1.train_iterate(5000, dataloader.get_train_batch, dataloader.get_val_batch)

print(dataloader.token_codec.decode(trainer1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), 100)[0].tolist()))

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dataloader.token_codec.decode(trainer1.generate(context, max_new_tokens=2000)[0].tolist()))

File already downloaded
in_size=32, in_embd=64, out_size=16, out_embd=64
in_size=16, in_embd=64, out_size=8, out_embd=64
in_size=8, in_embd=64, out_size=16, out_embd=64
in_size=16, in_embd=64, out_size=32, out_embd=64
0.414218 M parameters
step 0: train loss 15.9134, val loss 15.9326
step 100: train loss 3.5526, val loss 3.5994
step 200: train loss 3.4700, val loss 3.5172
step 300: train loss 3.4212, val loss 3.4761
step 400: train loss 3.3716, val loss 3.4224
step 500: train loss 3.3541, val loss 3.3911
step 600: train loss 3.3390, val loss 3.3761
step 700: train loss 3.3235, val loss 3.3652
step 800: train loss 3.2723, val loss 3.3054
step 900: train loss 3.1570, val loss 3.1782
step 1000: train loss 2.6906, val loss 2.7285
step 1100: train loss 2.5814, val loss 2.6122
step 1200: train loss 2.5668, val loss 2.5776
step 1300: train loss 2.5437, val loss 2.5688
step 1400: train loss 2.5383, val loss 2.5566
step 1500: train loss 2.5172, val loss 2.5416
step 1600: train loss 2.5017, val 

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.

In [3]:

#print(dataloader.token_codec.decode(trainer1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), 100)[0].tolist()))

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dataloader.token_codec.decode(trainer1.generate(context, max_new_tokens=2000)[0].tolist()))

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.

In [4]:
print(dataloader.token_codec.decode(trainer1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), 100)[0].tolist()))


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.

In [7]:
trainer1.predict("abc")

'bca;S;A:d Im uhOehwhtohortrrt rn'

In [11]:
idx=torch.zeros((1, 1), dtype=torch.long, device=device)
block_size = 32
max_new_tokens=100
for _ in range(max_new_tokens):
    # crop idx to the last block_size tokens
    idx_cond = idx[:, -block_size:]
    # get the predictions
    logits = trainer1.runner.forward(idx_cond)
    # focus only on the last time step
    logits = logits[:, -1, :]  # becomes (B, C)
    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1)  # (B, C)
    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
    # append sampled index to the running sequence
    idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
return idx

NameError: name 'F' is not defined